# GEOS 505 Research Computing in Earth System Science 
# Final Project: Progress Update
***

### D. Kody Johnson                                  
### November 29, 2018
---

## Introduction & Background

In the previous submittal (Dataset Overview), two scripts were developed. The first script comprised a set of ABAQUS commands to conduct a parametric study of the strain response of an AREMA 132R rail section to applied vertical and lateral loads. The vertical and lateral loads were varied for each simulation in an effort to find the strain (based on nodal deformations) of eight (8) ‘virtual’ strain gauges. The corresponding nodal deformations were then output to an output file (.dat file) for post processing to calculate the respective shear strains.

In the second script, the differential shear strain due to the applied loading for a single simulation was determined. The shear strain was then related to the applied loading through material and geometric properties of the rail section according to the differential shear theory presented in the Problem Statement submittal. This step was a validation that the nodal deformations were properly used to correctly calculate the principle strains  and differential shear. 

## Updated Workflow

In this submittal, the script developed for a single simulation was expanded to iterate through all simulation output files to calculate the eight (8) corresponding principle strain values. These values represent the features or predictors of a training data set used as the input to a machine learning algorithm. Each sample of this training dataset corresponds to each of the simulations in which both horizontal and vertical load were varied from zero (0) load to 250 kN. The load combinations of vertical and lateral load used in each simulation are the response variables in the training dataset. The training data will be used in the next submittal to determine two mapping functions: one for the vertical load and one for the lateral load. 

The following code gathers the nodal deformations corresponding to each strain gauge from the simulation output data files, validates that the principle strains are properly calculated, and constructs an array of vertical load, lateral load, and the eight strain values. Note: the code has been commented extensively to describe the various steps carried out in the script. 

In [6]:
#This script takes coordinate and displacement data for nodes representing
#   virtual strain gauges on a small section of AREMA 132RE rail and calculates
#   the infinitesimal strain that would be measured by these strain gauges due
#   to applied vertical and lateral loads. 

#import necessary libraries
import os    #
import numpy as np
import re
#import sklearn.gaussian_process as GP #These libraries will be used in the 
#import sklearn.linear_model as LM     #   next submittal

# Import original coordinates to an ndarray. All 'gauge' nodes are within
#   first 50 nodes in the file (as numbered by ABAQUS). Not all 50 are used, and
#   they must be reorganized into a consistent pattern for calculation. The 
#   conventioned used is as follows: starting at location A on the right side,
#   the first node is the upper right node of th gauge element and the second
#   node is the lower left (+45 degrees). The third node is the lower right and
#   the fourth node is the upper left (-45 degrees). This pattern is repeated 
#   for location B on the right side, then location A on the left side and 
#   finally location B on the left side (16 nodes in total).

with open('orig_coords.txt') as f:
    lines = f.readlines()
OC = np.zeros([len(lines),4],dtype=float)    #Original Coordinate array
for i in range(0,len(lines),1):
    OC[i] = np.fromstring(lines[i],dtype=float,sep = ",")
OC = np.vstack(OC)

# Reorganize the nodes into pattern described above

OP = np.zeros([16,3],dtype=float)    #Original Point array
for i in range(0,50,1):
    if OC[i,0] == 49.:
        OP[0,:] = OC[i,1:4]
    elif OC[i,0] == 15.:
        OP[1,:] = OC[i,1:4]
    elif OC[i,0] == 8.:
        OP[2,:] = OC[i,1:4]
    elif OC[i,0] == 45.:
        OP[3,:] = OC[i,1:4]
    elif OC[i,0] == 37.:
        OP[4,:] = OC[i,1:4]
    elif OC[i,0] == 20.:
        OP[5,:] = OC[i,1:4]
    elif OC[i,0] == 16.:
        OP[6,:] = OC[i,1:4] 
    elif OC[i,0] == 40.:
        OP[7,:] = OC[i,1:4]
    elif OC[i,0] == 50.:
        OP[8,:] = OC[i,1:4]
    elif OC[i,0] == 14.:
        OP[9,:] = OC[i,1:4]
    elif OC[i,0] == 5.:
        OP[10,:] = OC[i,1:4]
    elif OC[i,0] == 46.:
        OP[11,:] = OC[i,1:4]
    elif OC[i,0] == 38.:
        OP[12,:] = OC[i,1:4]
    elif OC[i,0] == 19.:
        OP[13,:] = OC[i,1:4]
    elif OC[i,0] == 13.:
        OP[14,:] = OC[i,1:4]
    elif OC[i,0] == 39.:
        OP[15,:] = OC[i,1:4]      

#Create a list of .dat file names to be used with open()

file_list = [] #Initialize filer list.
folder = [] #Initialize folder list.
for dirName, subdirList, fileList in os.walk('.'): #Use OS.walk to walk each folder and directory and create a path name     
    for fname in fileList:                         #to each file.
        if fname.endswith(".dat"): 
           path = dirName + '\\' + fname          #Build windows readable path neam.
           file_list = file_list + [path]


# Read and organize the file_list array in the same order the ABAQUS parametric
#   study used (defined in the 'design_definitions.txt' file) OS walk opens the 
#   files in alphanumeric order and so the design number (i.e. c1, c2,... c36)  
#   in the file name is used to relate the order of the file_list list to the  
#   order defined in the 'design_definitions.txt' file.
           
des = np.zeros([len(file_list),1],dtype=int)    #design array (desing number)
pat1 = re.compile(r"_c")          
for i in range (0,len(file_list),1):
    if pat1.search(file_list[i]) != None:   #Only open .dat files with _c in 
        pat2 = re.compile(r'\d+')           #   the name. This step is not
        desn = pat2.findall(file_list[i])   #   necessary as all other .dat 
        des[i,0] = int(desn[0])             #   files have been removed

# The des array is a list of design numbers (e.g. c1, c2 etc.) read from the 
#   file_list array. The next section uses this array to reorganize the 
#   file_list 

des = np.vstack(des[:,0].argsort())
count = -1
flist = np.vstack([None]*len(file_list))   #Ordered file_list array
for ind in des:
    count = count + 1
    flist[count] = file_list[ind[0]]
flist = flist.flatten()
flist = flist.tolist()

# Using the flist array, open each file and read the displacement data. The 
#   .dat files have 3/4 of a million lines of code and the location of the 
#   nodal displacement data may not be at the same location. Therefore, a 
#   regular expresson is used to search for a line of text preceeding the 
#   section in the .dat files that contains the displacement data. Once read,
#   the data is reogainzed in the same order as the OC (original coordinate)
#   array.     
   
U = np.zeros([16,3,len(flist)],dtype=float)  # Ordered Displacements (U)      
CD = np.zeros([16,4],dtype=float)     # Unordered Coordinate Displacements (CD)
for i in range(0,len(flist),1):
    with open(flist[i]) as f:
        lines = f.readlines()
        pat = re.compile(r"\bINCREMENT     6 SUMMARY\b")
        for j in range(0,len(lines),1):
            if pat.search(lines[j]) != None:
                disp = lines[j+19:j+35]    #Temporary displacement array (disp)
        for k in range(0,len(disp),1):
            CD[k] = np.fromstring(disp[k],dtype=float,sep = " ")
        CD = np.vstack(CD)
        for p in range(0,16,1):
            if CD[p,0] == 49.:
                U[0,:,i] = CD[p,1:4]
            elif CD[p,0] == 15.:
                U[1,:,i] = CD[p,1:4]
            elif CD[p,0] == 8.:
                U[2,:,i] = CD[p,1:4]
            elif CD[p,0] == 45.:
                U[3,:,i] = CD[p,1:4]
            elif CD[p,0] == 37.:
                U[4,:,i] = CD[p,1:4]
            elif CD[p,0] == 20.:
                U[5,:,i] = CD[p,1:4]
            elif CD[p,0] == 16.:
                U[6,:,i] = CD[p,1:4] 
            elif CD[p,0] == 40.:
                U[7,:,i] = CD[p,1:4]
            elif CD[p,0] == 50.:
                U[8,:,i] = CD[p,1:4]
            elif CD[p,0] == 14.:
                U[9,:,i] = CD[p,1:4]
            elif CD[p,0] == 5.:
                U[10,:,i] = CD[p,1:4]
            elif CD[p,0] == 46.:
                U[11,:,i] = CD[p,1:4]
            elif CD[p,0] == 38.:
                U[12,:,i] = CD[p,1:4]
            elif CD[p,0] == 19.:
                U[13,:,i] = CD[p,1:4]
            elif CD[p,0] == 13.:
                U[14,:,i] = CD[p,1:4]
            elif CD[p,0] == 39.:
                U[15,:,i] = CD[p,1:4] 
        
# Calculate the infinitesimal strain of each 'virtual' strain gauge (as
#   previously defined). This is done by first creating a Deformed Point (DP)
#   in which is the displacement is added to the original point. Using this 
#   set of points and the OC set of points, two sets of lines are created:
#   Original line (OL) and Deformed Line (DL). These lines are then used to 
#   calculate infinitesimal strain according to the strain definition.

ST = np.zeros([8,len(flist)],dtype=float)   # Strain array
P = np.zeros([len(flist),1],dtype=float)    # Estimated vertical load (to be
for i in range(0,len(flist),1):             #   described later)
    DP = np.zeros([16,3],dtype=float)   #Deformed point
    DP = OP + U[:,:,i]                          
    
    OL = np.zeros([8,1],dtype=float)    #Original line
    DL = np.zeros([8,1],dtype=float)    #Deformed line
    
    for j in range(0,8,1):
        OL[j,0] = np.linalg.norm(OP[2*j+1,:]-OP[2*j,:])
        DL[j,0] = np.linalg.norm(DP[2*j+1,:]-DP[2*j,:])
    strain = (DL - OL)/OL    #Definition of infitesimal strain.
    ST[:,i] = strain.reshape(8,)
  
# As a check to ensure that all strains were calculated correctly, the vertical
#   load P, estimated using the differential shear approach (described else-
#   where), is calculated and compared to the applied loads. Note: the first 6
#   elements of P correspond to cases in which the lateral load is zero; it is
#   only for these cases that the calculation will provide accurate results as
#   crosstalk from the lateral load causes an error in the differential shear 
#   calculation. 

# Principle strains on the right side (ea, eb, ec, ed; see Figure 1)
    
    ea = ST[0]
    eb = ST[1]
    ec = ST[2]
    ed = ST[3]
    eap = ST[4]
    ebp = ST[5]
    ecp = ST[6]
    edp = ST[7]
    
    e1 = (eb + ebp)/2 - (ea + eap)/2
    e2 = (ec + ecp)/2 - (ed + edp)/2
          
    delG = e1 + e2
    
    E = 2.07E+11
    I = 3.661E-05
    t = 0.0173795
    Q = 0.000258773
    v = 0.30
    
    
    P[i,0] = E*I*t/(2*Q*(1+v))*delG[i]
    
vert=[]
lat=[]
pat = re.compile(r'\d+')
with open('design_definitions.txt') as f:
    lines = f.readlines()
    for i in range(0,len(lines),1):
       line = lines[i]
       loads = pat.findall(line)
       vert = vert + [float(loads[1])]
       lat = lat + [float(loads[3])]
    loads = np.column_stack((vert,lat))
    ST = np.transpose(ST)
    #tr_data = np.hstack((loads,ST))   #convenient for copy/paste to other platforms.
    X = ST
    y = loads
    print(P[0:6,:])
    print('\n')
    print(X)
    print('\n')
    print(y)
    
    

[[     0.        ]
 [ 50322.60875946]
 [100646.76858273]
 [150972.02216487]
 [201298.64769655]
 [251626.12337242]]


[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-6.36416530e-05  6.48792191e-05  6.48553059e-05 -6.36542249e-05
  -6.36929522e-05  6.48632837e-05  6.48671253e-05 -6.36823365e-05]
 [-1.27171936e-04  1.29875771e-04  1.29822288e-04 -1.27197574e-04
  -1.27275623e-04  1.29840040e-04  1.29848601e-04 -1.27256216e-04]
 [-1.90591954e-04  1.94984635e-04  1.94902920e-04 -1.90631523e-04
  -1.90746367e-04  1.94933616e-04  1.94941033e-04 -1.90719122e-04]
 [-2.53899564e-04  2.60209539e-04  2.60095938e-04 -2.53957057e-04
  -2.54105793e-04  2.60138625e-04  2.60148263e-04 -2.54073531e-04]
 [-3.17094359e-04  3.25547955e-04  3.25401818e-04 -3.17171522e-04
  -3.17352372e-04  3.25462152e-04  3.25467037e-04 -3.17316919e-04]
 [ 3.16649020e-04 -1.03112516e-06 -1.03628151e-05  2.89240714e-04
  -2.36472391e-04  